In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/R&D/Evaluation/xlwic_wn

/content/drive/MyDrive/Colab Notebooks/R&D/Evaluation/xlwic_wn


In [4]:
import spacy
import json
from tqdm import tqdm

## dev_de-de

In [ ]:
!python -m spacy download de

In [2]:
def preprocess(infile):
  with open(infile, encoding='utf-8') as data_file:
    data = data_file.readlines()
  pairs = []
  i = 0
  for line in tqdm(data):
    pair_id = 'id_' + str(i)
    target_lemma, target_POS, start_1, end_1, start_2, end_2, sentence_1, sentence_2, label = line.split('\t')
    start_1 = int(start_1)
    end_1 = int(end_1)
    start_2 = int(start_2)
    end_2 = int(end_2)
    label = int(label)
    language_1 = infile.split('.')[1].split('-')[0]
    language_2 = infile.split('.')[1].split('-')[1]
    target_1 = sentence_1[start_1:end_1]
    target_2 = sentence_2[start_2:end_2]
    pairs.append((pair_id, target_lemma, target_POS, language_1, language_2, sentence_1, sentence_2, target_1, target_2, label))
    i += 1
  return pairs

In [6]:
def pairs_to_conllu(pairs, outfile):
  result = open(outfile, "a")
  for pair in tqdm(pairs):
    pair_id = pair[0]
    target_lemma = pair[1]
    target_POS = pair[2]
    language_1 = pair[3]
    language_2 = pair[4]
    sentence_1 = pair[5]
    sentence_2 = pair[6]
    target_1 = pair[7]
    target_2 = pair[8]
    label = pair[9]

    # write comment of pair_id
    result.write('#\tpair_id\t=\t')
    result.write(pair_id)
    result.write('\n')
    # write comment of target
    result.write('#\ttarget_lemma\t=\t')
    result.write(target_lemma)
    result.write('\n')
    result.write('#\ttarget_POS\t=\t')
    result.write(target_POS)
    result.write('\n')
    
    result.write('\n')
    # write comment of first sentence
    result.write('#\tfirst_sentence\t=\t')
    result.write(sentence_1)
    result.write('\n')
    # write one token per line of first text
    if language_1 != 'zh' and language_1 != 'da' and language_1 != 'ru':
      nlp_1 = spacy.load(language_1)
    elif language_1 == 'zh':
      nlp_1 = spacy.load(language_1+'_core_web_sm')
    else:
      nlp_1 = spacy.load(language_1+'_core_news_sm') 
    doc_1 = nlp_1(sentence_1)
    count = 0
    for token in doc_1:
      result.write(str(count)) # INDEX
      result.write('\t')
      result.write(token.text) # FORM
      result.write('\t')
      result.write(token.lemma_) # LEMMA
      result.write('\t')
      result.write(token.pos_) # UPOS
      result.write('\t')
      result.write(token.head.text) # HEAD
      result.write('\t')
      result.write(token.dep_) # DEPREL
      result.write('\t')
      result.write(str(token.text==target_1)) # IS TARGET
      result.write('\n')
      count += 1
    
    result.write('\n')
    # write comment of second sentence
    result.write('#\tsecond_sentence\t=\t')
    result.write(sentence_2)
    result.write('\n')
    # write one token per line of first text
    if language_2 != 'zh' and language_2 != 'da' and language_2 != 'ru':
      nlp_2 = spacy.load(language_2)
    elif language_2 == 'zh':
      nlp_2 = spacy.load(language_2+'_core_web_sm')
    else:
      nlp_2 = spacy.load(language_2+'_core_news_sm')
    doc_2 = nlp_2(sentence_2)
    count = 0
    for token in doc_2:
      result.write(str(count)) # INDEX
      result.write('\t')
      result.write(token.text) # FORM
      result.write('\t')
      result.write(token.lemma_) # LEMMA
      result.write('\t')
      result.write(token.pos_) # UPOS
      result.write('\t')
      result.write(token.head.text) # HEAD
      result.write('\t')
      result.write(token.dep_) # DEPREL
      result.write('\t')
      result.write(str(token.text==target_2)) # IS TARGET
      result.write('\n')
      count += 1
    
    result.write('\n')
    # write label
    result.write('#\tlabel\t=\t')
    result.write(str(label))
    result.write('\n')
    result.write('\n\n')

  result.close()

In [ ]:
de_de_pairs = preprocess('dev.de-de.txt')
pairs_to_conllu(de_de_pairs, 'dev.de-de.conllu')

100%|██████████| 1000/1000 [49:04<00:00,  2.94s/it]


In [7]:
def find_info(sentence_pair):
 
  basic_info, sent_1, sent_2, label_info = sentence_pair.split('\n\n')
  info = []

  for line in basic_info.split('\n'):
      breakdown = line.split('\t')
      if breakdown[1] == 'pair_id':
          pair_id = breakdown[3]
  
  info.append(pair_id)
  
  for sent in (sent_1, sent_2):
      target = []
      head = []
      dependent = []
      for line in sent.split('\n'):
          breakdown = line.split('\t')
          if len(breakdown) == 4:
              info.append(breakdown[-1])
          elif (len(breakdown) > 4) and breakdown[6] == 'True':
              target.append(breakdown[1])
              head.append(breakdown[4])

      # Find dependent  
      for line in sent.split('\n'):
          breakdown = line.split('\t')
          if (len(breakdown) > 4) and breakdown[4] in target:
              dependent.append(breakdown[1])
    
      info.append(target)
      info.append(head)
      info.append(dependent)
    
  for line in label_info.split('\n'):
    breakdown = line.split('\t')
    if breakdown[1] == 'label':
      label = breakdown[3]

  info.append(int(label))
  

  return info

In [8]:
def conllu_to_info(conllu_file):
  prefix = conllu_file[:-7]
  
  con_file = open(conllu_file, 'r')
  infos = open(prefix+'.info', "a", encoding='utf8')
  i = 0
  for sentence_pair in tqdm(con_file.read().split('\n\n\n')[:-1]):
    info = find_info(sentence_pair)
    #infos.write(json.dumps(info))
    json.dump(info, infos, ensure_ascii=False)
    infos.write('\n')
    i += 1

In [ ]:
conllu_to_info('dev.de-de.conllu')

100%|██████████| 1000/1000 [00:00<00:00, 8560.80it/s]


## dev_da-da

In [ ]:
!pip uninstall spacy

In [ ]:
!pip install spacy==2.3.0

In [ ]:
!python -m spacy download da_core_news_sm

In [ ]:
da_da_pairs = preprocess('dev.da-da.txt')
pairs_to_conllu(da_da_pairs, 'dev.da-da.conllu')

100%|██████████| 500/500 [40:14<00:00,  4.83s/it]


In [ ]:
conllu_to_info('dev.da-da.conllu')

100%|██████████| 500/500 [00:00<00:00, 9070.22it/s]


## dev_en-en

In [ ]:
!pip uninstall spacy

In [ ]:
!pip install spacy==2.3.0

In [ ]:
!python -m spacy download en

In [ ]:
en_en_pairs = preprocess('dev.en-en.txt')
pairs_to_conllu(en_en_pairs, 'dev.en-en.conllu')

100%|██████████| 500/500 [11:09<00:00,  1.34s/it]


In [ ]:
conllu_to_info('dev.en-en.conllu')

100%|██████████| 500/500 [00:00<00:00, 15616.36it/s]


## dev_fr-fr

In [ ]:
!python -m spacy download fr

In [ ]:
fr_fr_pairs = preprocess('dev.fr-fr.txt')
pairs_to_conllu(fr_fr_pairs, 'dev.fr-fr.conllu')

100%|██████████| 500/500 [22:16<00:00,  2.67s/it]


In [ ]:
conllu_to_info('dev.fr-fr.conllu')

100%|██████████| 500/500 [00:00<00:00, 9810.04it/s]


## dev_zh-zh

In [ ]:
!pip uninstall spacy

In [ ]:
!pip install spacy==2.3.1

In [ ]:
!python -m spacy download zh_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('zh_core_web_sm')


In [ ]:
zh_zh_pairs = preprocess('dev.zh-zh.txt')
pairs_to_conllu(zh_zh_pairs, 'dev.zh-zh.conllu')

  0%|          | 0/500 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.775 seconds.
Prefix dict has been built successfully.
100%|██████████| 500/500 [23:30<00:00,  2.82s/it]


In [ ]:
conllu_to_info('dev.zh-zh.conllu')

100%|██████████| 500/500 [00:00<00:00, 8724.86it/s]
